In [1]:
# a bit of setup
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import animation
import pandas as pd
import scipy.stats as stats
from pydub import AudioSegment
% matplotlib inline

In [4]:
def metropolis_hastings(proposal_func, init_func, acceptance_score, num_iters, step=30):
    """
    Runs the metropolis-hastings algorithm for
    num_iters iterations, using proposal_func
    to generate samples and scorer to assign
    probability scores to samples.
      
    proposal_func -- function that proposes
        candidate state; takes in current state as
        argument and returns candidate state
    init_func -- function that proposes starting
        state; takes no arguments and returns a
        sample state
    acceptance_score -- function that calculates the acceptance
        probability; takes in two state samples
        (candidate first, then sample) and returns
        acceptance probability
    
    Returns a sequence of every step-th sample. You 
    should only sample on upon acceptance of a new
    proposal. Do not keep sampling the current state.
    
    Note the total number of samples will NOT be
    equal to num_iters. num_iters is the total number
    of proposals we generate.
    """
    samples = []
    sample = init_func()
    for i in range(num_iters):
        candidate = proposal_func(sample)
        acceptance_ratio = min(1, acceptance_score(candidate,sample))
        if np.random.uniform() < acceptance_ratio:
            sample = candidate
            samples.append(sample)
    return samples[::step]

In [13]:
def build_bigram_freq_matrix(input_arr, alphabet_size):
    """
    Builds a matrix that represents the transitional
    probabilities between letters in input_file.
    
    bigram_freq_matrix[0][1] is the probability of
    transitioning from the 0th letter of the alphabet
    to the 1st letter of the alphabet, where letters
    are zero-indexed. ' ' (space) is denoted as the
    26th letter of the alphabet.
    """
    counts = np.ones([alphabet_size, alphabet_size])
    
    for i in range(len(input_arr) - 2):
        first_char = input_arr[i]
        second_char = input_arr[i+1]
        counts[first_char][second_char] += 1
        
    return (counts.T / np.sum(counts, axis=1)).T

In [3]:
def starting_state():
    """
    Start with a random permutation.
    """
    return np.random.randint(1,21,21)

def sample_candidate(sample):
    """
    To search for new ciphers, randomly
    swap two letters in the previous cipher.
    """
    num_permute = 3 #hyperparameter
    sample = list(sample)
    for _ in range(num_permute):
        sample[np.random.randint(1,21)] = np.random.randint(1,21)
    return sample
    

def make_acceptance_scorer(transition_matrix):
    """
    Calculate the acceptance probability, which is the
    probability of observing the message translated by
    the proposed cipher divided by the probability of
    obseving the message translated by the current
    cipher.
    """
    
    def scorer(candidate, sample):
        nonlocal transition_matrix
        
        p_cand = 1
        for i in range(len(candidate) - 1):
            p_cand *= transition_matrix[candidate[i]][candidate[i + 1]]
            
        p_samp = 1
        for i in range(len(samp) - 1):
            p_samp *= transition_matrix[samp[i]][samp[i + 1]]
            
        return max(1, p_cand / p_samp)
    
    return scorer